In [ ]:
!pip install backtrader
!pip install pyfolio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.1/410.1 KB 12.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.3 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88685 sha256=630e080e8f0e7b5ef04472780c953dc16722545628f80a38190bd4fd1643dabd
  Stored in directory: /root/.cache/pip/wheels/75/c7/b9/986a1ba8adb2afce230e1a497dc11aeb33efef4fc8dfd6c649
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39779 sha256=e0ee108e00c38a923de1cc35507e3c3e11eb44e41427339cf6

In [ ]:
import backtrader as bt 
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import datetime
import pandas as pd

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
weekly_path = '/content/drive/My Drive/IRAGE/9.22/16-22-SEP-WEEKLY-expiry_data_VEGE_NF_AND_BNF_Options_Vege.csv'
monthly_path = '/content/drive/My Drive/IRAGE/9.22/SEPTEMBER-2022-MONTHLY-Expiry-data _Vege.csv'
start_date = pd.read_csv(weekly_path)["Date/Time"].min().split(" ")[0]
end_date = pd.read_csv(weekly_path)["Date/Time"].max().split(" ")[0]

In [ ]:
class My_strategy(bt.Strategy):

    def __init__(self):
        self.order = None
        self.mapping = {}
        for i, _data in enumerate(self.datas):
            self.mapping[_data._name] = i
        self.leg = 0
        self.initiate = 0
        self.notyet = False
        self.firsttwo = 1
        self.upper = 0 #variable if atm travels up by delta
        self.lower = 0  #variable if atm travels down by delta
        self.delta = 50
        self.stake = 25 
        self.history = [] #list storing all open positions
        self.master = []

        print(self.mapping)

    def log(self, txt, doprint=True):
        date = self.data.datetime.date(0)
        time = self.data.datetime.time(0)
        if doprint:
            print(str(date) + '--' + txt)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        elif order.status == order.Completed:
            # if order.isbuy():
            #     self.log('BUY ticker: {}  price: {}, value: {}, commission: {}'.format(order.executed.name, order.executed.price, order.executed.value, order.executed.comm))
            # if order.issell():
            #     self.log('SELL ticker: {}  price: {}, value: {}, commission: {}'.format(order.executed.name, order.executed.price, order.executed.value, order.executed.comm))
            if order.isbuy():
                self.log('BUY  price: {}, value: {}, commission: {}'.format(order.executed.price, order.executed.value, order.executed.comm))
            if order.issell():
                self.log('SELL  price: {}, value: {}, commission: {}'.format(order.executed.price, order.executed.value, order.executed.comm))
        elif order.status in [order.Rejected, order.Margin]:
            self.log('Order Rejected/Margin')

        self.order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            self.log('CLOSE   Gross P/L: {}, Net P/L: {}'.format(trade.pnl, trade.pnlcomm))

    def bearish(self,name,stk):
        datanamenear = name + "WK" + str(stk) + "PE"
        datanamefar = name + str(stk - 300) + "CE"
        if (datanamenear in self.mapping.keys()) and (datanamefar in self.mapping.keys()):
            self.sell(data = datanamenear, size = self.stake)
            self.sell(data = datanamefar, size = 2*self.stake)
            self.history.append([stk, [datanamenear, datanamefar]])
        else :
            print("data not found")
            self.leg = 0  #leg is not changed if its data is not found
        return 

    def bullish(self,name,stk):
        datanamenear = name + "WK" + str(stk) + "CE"
        datanamefar = name + str(stk + 300) + "PE"
        if (datanamenear in self.mapping.keys()) and (datanamefar in self.mapping.keys()):
            self.sell(data = datanamenear, size = self.stake)
            self.sell(data = datanamefar, size = 2*self.stake)
            self.history.append([stk, [datanamenear, datanamefar]])
        else :
            print("data not found")
            self.leg = 1     #leg is not changed if its data is not found
        return 

    def strike_price(self, price):
        return round(price/100)*100

    def sign_change(self, value1, value2):
        if (value1*value2 > 0):
            return 1 
        else :
            return -1

    def next(self):
        stk = self.strike_price(self.datas[self.mapping["BANKNIFTY"]].close[0])
        date = self.data.datetime.date()
        time = self.data.datetime.time()
        # print(time)
        datetime = str(date) + " " + str(time)
        datetime = pd.to_datetime(datetime)
        print(datetime)
        print(self.datas[self.mapping["BANKNIFTY"]].close[0])
        print(self.history)
        print("upper", self.upper)
        print("lower", self.lower)
        # print("positions")
        # print(self.getpositions().__doc__) 
        # print(type("10:00:00"))
        name = "BANKNIFTY"
        atm = self.datas[self.mapping["BANKNIFTY"]].close[0]
        # for data in self.datas:
        #     if self.getposition(data).size:
        #         print(data._name)
        #         print(self.getposition(data))
        if str(time) == "10:00:00":
            print("day started")
        if self.initiate == 0 and str(time) == "10:00:00":             #initiate to make the first trade get executedd on the open price
            atm = self.datas[self.mapping["BANKNIFTY"]].open[0]
            self.upper = atm + self.delta
            self.lower = atm - self.delta
            self.bearish(name,stk)
            self.initiate = 1
            self.leg = 1
            self.notyet = True

        elif (str(time) == "15:00:00"):                                #for the last trade of the day
            for s in reversed(self.history) :
                self.buy(data = s[1][0], size = self.stake)
                self.buy(data = s[1][1], size = 2*self.stake)
                self.history.pop(-1)
            self.initiate = 0                                      #resetting all parameters at the end of the day
            self.leg = 0 
            self.notyet = False
            self.firsttwo = 1
            self.upper = 0
            self.lower = 0
            print("Done for the Day!")
            print(self.history)
            print(self.positions)


        elif (self.notyet == True):
            if (atm >= self.upper or atm <= self.lower):
                trend_change = -1
                if self.firsttwo >= 2 :
                    prev_change = self.history[-1][0] - self.history[-2][0]  #to offset the errors from this line for the first two trades we made the firsttwo variable
                    trend_change = self.sign_change(stk - self.history[-1][0] , prev_change)

                if trend_change == 1 or self.firsttwo < 2 :            #for no trend reversal
                    if self.leg == 0 :                   
                        self.bearish(name,stk)
                        self.leg = 1                                   #changing legs
                    else :
                        self.bullish(name,stk)
                        self.leg = 0                                   #changing legs
                    self.upper = atm + self.delta
                    self.lower = atm - self.delta
                    self.firsttwo += 1

                else :                                                 #for trend reversal
                    for s in reversed(self.history) :                  #doing it in reverse iterartions
                        self.buy(data = s[1][0], size = self.stake)
                        self.buy(data = s[1][1], size = 2*self.stake)  #exit positions
                        self.history.pop(-1)                           #delete the trade from open positions
                        if s[0] == stk:      
                            # print("removed")
                            # print(self.history)
                            # print(self.getposition("BANKNIFTYWK40900PE"))
                            break
                      
                    if len(self.history) == 0 :
                        self.firsttwo = 1
                    if self.leg == 0 :
                        self.bearish(name,stk)
                        self.leg = 1
                    else :
                        self.bullish(name,stk)
                        self.leg = 0
                    self.upper = atm + self.delta
                    self.lower = atm - self.delta
                  

        else :
            print("Not Yet!")


In [ ]:
from backtrader.feeds import PandasData

OHLCV = ['open', 'high', 'low', 'close', 'volume']

class SignalData(PandasData):
    """
    Define pandas DataFrame structure
    """
    cols = OHLCV

    # create lines
    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

In [ ]:
# import os
# import pandas as pd
# import numpy as np
# import datetime
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def convert_datetime(datetime_str):
  _format = '%d-%m-%Y %H:%M:%S'
  return datetime.datetime.strptime(datetime_str, _format)

In [ ]:
filename = weekly_path
df = pd.read_csv(filename)
df = df[df['Ticker'].str.contains('BANK') & df['Ticker'].str.contains('E')]
df = df.assign(strike = lambda row: row['Ticker'])
df['strike'] = df.strike.apply(lambda x: int(x[-7:-2]))
df['datetime'] = pd.to_datetime(df['Date/Time'])

In [ ]:
df

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest,strike,datetime
3752,BANKNIFTYWK30000PE,16-09-2022 09:15:00,2.20,2.20,2.00,2.00,525,18325,30000,2022-09-16 09:15:00
3753,BANKNIFTYWK30000PE,16-09-2022 09:16:00,2.00,2.10,2.00,2.05,3625,18325,30000,2022-09-16 09:16:00
3754,BANKNIFTYWK30000PE,16-09-2022 09:17:00,2.05,2.25,2.05,2.15,11525,21750,30000,2022-09-16 09:17:00
3755,BANKNIFTYWK30000PE,16-09-2022 09:18:00,2.15,2.35,2.15,2.20,26850,21750,30000,2022-09-16 09:18:00
3756,BANKNIFTYWK30000PE,16-09-2022 09:19:00,2.20,2.20,2.00,2.05,16800,21750,30000,2022-09-16 09:19:00
...,...,...,...,...,...,...,...,...,...,...
206529,BANKNIFTYWK46000CE,22-09-2022 15:25:00,0.05,0.05,0.05,0.05,350,179300,46000,2022-09-22 15:25:00
206530,BANKNIFTYWK46000CE,22-09-2022 15:26:00,0.05,0.05,0.05,0.05,425,179300,46000,2022-09-22 15:26:00
206531,BANKNIFTYWK46000CE,22-09-2022 15:27:00,0.05,0.05,0.05,0.05,25,178525,46000,2022-09-22 15:27:00
206532,BANKNIFTYWK46000CE,22-09-2022 15:28:00,0.05,0.15,0.05,0.15,1025,178525,46000,2022-09-22 15:28:00


In [ ]:
df['Ticker'].unique()

array(['BANKNIFTYWK30000PE', 'BANKNIFTYWK30500PE', 'BANKNIFTYWK31000CE',
       'BANKNIFTYWK31000PE', 'BANKNIFTYWK31500PE', 'BANKNIFTYWK32000PE',
       'BANKNIFTYWK32500PE', 'BANKNIFTYWK33000CE', 'BANKNIFTYWK33000PE',
       'BANKNIFTYWK33500CE', 'BANKNIFTYWK33500PE', 'BANKNIFTYWK34000PE',
       'BANKNIFTYWK34500CE', 'BANKNIFTYWK34500PE', 'BANKNIFTYWK35000CE',
       'BANKNIFTYWK35000PE', 'BANKNIFTYWK35500CE', 'BANKNIFTYWK35500PE',
       'BANKNIFTYWK35700PE', 'BANKNIFTYWK35800PE', 'BANKNIFTYWK35900PE',
       'BANKNIFTYWK36000CE', 'BANKNIFTYWK36000PE', 'BANKNIFTYWK36100PE',
       'BANKNIFTYWK36200PE', 'BANKNIFTYWK36300PE', 'BANKNIFTYWK36400PE',
       'BANKNIFTYWK36500CE', 'BANKNIFTYWK36500PE', 'BANKNIFTYWK36600PE',
       'BANKNIFTYWK36700CE', 'BANKNIFTYWK36700PE', 'BANKNIFTYWK36800CE',
       'BANKNIFTYWK36800PE', 'BANKNIFTYWK36900CE', 'BANKNIFTYWK36900PE',
       'BANKNIFTYWK37000CE', 'BANKNIFTYWK37000PE', 'BANKNIFTYWK37100PE',
       'BANKNIFTYWK37200CE', 'BANKNIFTYWK37200PE', 

In [ ]:
df.sort_index()

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest,strike,datetime
3752,BANKNIFTYWK30000PE,16-09-2022 09:15:00,2.20,2.20,2.00,2.00,525,18325,30000,2022-09-16 09:15:00
3753,BANKNIFTYWK30000PE,16-09-2022 09:16:00,2.00,2.10,2.00,2.05,3625,18325,30000,2022-09-16 09:16:00
3754,BANKNIFTYWK30000PE,16-09-2022 09:17:00,2.05,2.25,2.05,2.15,11525,21750,30000,2022-09-16 09:17:00
3755,BANKNIFTYWK30000PE,16-09-2022 09:18:00,2.15,2.35,2.15,2.20,26850,21750,30000,2022-09-16 09:18:00
3756,BANKNIFTYWK30000PE,16-09-2022 09:19:00,2.20,2.20,2.00,2.05,16800,21750,30000,2022-09-16 09:19:00
...,...,...,...,...,...,...,...,...,...,...
206529,BANKNIFTYWK46000CE,22-09-2022 15:25:00,0.05,0.05,0.05,0.05,350,179300,46000,2022-09-22 15:25:00
206530,BANKNIFTYWK46000CE,22-09-2022 15:26:00,0.05,0.05,0.05,0.05,425,179300,46000,2022-09-22 15:26:00
206531,BANKNIFTYWK46000CE,22-09-2022 15:27:00,0.05,0.05,0.05,0.05,25,178525,46000,2022-09-22 15:27:00
206532,BANKNIFTYWK46000CE,22-09-2022 15:28:00,0.05,0.15,0.05,0.15,1025,178525,46000,2022-09-22 15:28:00


In [ ]:
full_data = pd.DataFrame()
for day in pd.date_range(start_date , end_date):
  full_data = pd.concat([full_data, pd.DataFrame(pd.date_range(f"{day.date()} 09:15:00", f"{day.date()} 15:29:00", freq="1T"))])

full_data = full_data.reset_index(drop=True)
full_data.columns = ["datetime"]

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Parsing '16-09-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Parsing '22-09-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dfs = []
keys = []
raws = []
for key, group in df.groupby("Ticker"):
  group = pd.merge(full_data, group, how="left").fillna(method='ffill').fillna(method='bfill')
  if group.isna().sum().sum() == 0:
    keys.append(key)
    dfs.append(group.set_index("datetime").groupby(
      pd.Grouper(freq='15Min')).agg({
      "Open": "first",
      "High": "max",
      "Low": "min",
      "Close": "last",
      "Volume": "sum",
      "Open Interest": "last"}).dropna())
    raws.append(group)

In [ ]:
dfs[0].head()

,Open,High,Low,Close,Volume,Open Interest
datetime,,,,,,
2022-09-16 09:15:00,2.20,2.35,2.00,2.05,96500.0,33825.0
2022-09-16 09:30:00,2.00,2.20,2.00,2.10,148775.0,51900.0
2022-09-16 09:45:00,2.15,2.15,1.90,2.10,112550.0,55700.0
2022-09-16 10:00:00,2.05,2.10,2.00,2.05,69825.0,62150.0
2022-09-16 10:15:00,2.05,2.05,1.95,2.00,11875.0,63200.0


In [ ]:
filename2 = monthly_path
df2 = pd.read_csv(filename2)
spot_df = df2[df2['Ticker'] == 'BANKNIFTY']
df2 = df2[df2['Ticker'].str.contains('BANK') & df2['Ticker'].str.contains('E')]
df2 = df2.assign(strike = lambda row: row['Ticker'])
df2['strike'] = df2.strike.apply(lambda x: int(x[-7:-2]))
df2['datetime'] = pd.to_datetime(df2['Date/Time'])


In [ ]:
df2.sort_index()

,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest,strike,datetime
18019,BANKNIFTY30000CE,26-08-2022 15:29:00,9180.00,9180.00,9180.00,9180.00,25,19100,30000,2022-08-26 15:29:00
18020,BANKNIFTY30000CE,29-08-2022 10:15:00,8310.00,8310.00,8310.00,8310.00,25,19125,30000,2022-08-29 10:15:00
18021,BANKNIFTY30000CE,29-08-2022 15:29:00,8310.00,8310.00,8310.00,8310.00,25,19125,30000,2022-08-29 15:29:00
18022,BANKNIFTY30000CE,30-08-2022 11:14:00,8986.50,8986.50,8986.50,8986.50,25,19150,30000,2022-08-30 11:14:00
18023,BANKNIFTY30000CE,30-08-2022 12:08:00,8950.05,8950.05,8950.05,8950.05,25,19175,30000,2022-08-30 12:08:00
...,...,...,...,...,...,...,...,...,...,...
802417,BANKNIFTY46000PE,23-09-2022 09:17:00,5571.45,5571.45,5571.45,5571.45,50,100,46000,2022-09-23 09:17:00
802418,BANKNIFTY46000PE,23-09-2022 15:25:00,6449.90,6449.90,6449.90,6449.90,50,150,46000,2022-09-23 15:25:00
802419,BANKNIFTY46000PE,26-09-2022 09:16:00,6987.05,6987.05,6987.05,6987.05,50,150,46000,2022-09-26 09:16:00
802420,BANKNIFTY46000PE,26-09-2022 13:29:00,7165.85,7165.85,7165.85,7165.85,25,150,46000,2022-09-26 13:29:00


In [ ]:
full_data2 = pd.DataFrame()
for day in pd.date_range(start_date, end_date):
  full_data2 = pd.concat([full_data2, pd.DataFrame(pd.date_range(f"{day.date()} 09:15:00", f"{day.date()} 15:29:00", freq="1T"))])

print(full_data2)
full_data2 = full_data2.reset_index(drop=True)
full_data2.columns = ["datetime"]

                      0
0   2022-09-16 09:15:00
1   2022-09-16 09:16:00
2   2022-09-16 09:17:00
3   2022-09-16 09:18:00
4   2022-09-16 09:19:00
..                  ...
370 2022-09-22 15:25:00
371 2022-09-22 15:26:00
372 2022-09-22 15:27:00
373 2022-09-22 15:28:00
374 2022-09-22 15:29:00

[2625 rows x 1 columns]


/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Parsing '16-09-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Parsing '22-09-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# spot_data

In [ ]:
# spot_df

In [ ]:
dfs2 = []
keys2 = []
raws2 = []
for key2, group2 in df2.groupby("Ticker"):
  group2 = pd.merge(full_data2, group2, how="left").fillna(method='ffill').fillna(method='bfill')
  if group2.isna().sum().sum() == 0:
    keys2.append(key2)
    dfs2.append(group2.set_index("datetime").groupby(
      pd.Grouper(freq='15Min')).agg({
      "Open": "first",
      "High": "max",
      "Low": "min",
      "Close": "last",
      "Volume": "sum",
      "Open Interest": "last"}).dropna())
    raws2.append(group2)

In [ ]:
raws2[-2]

,datetime,Ticker,Date/Time,Open,High,Low,Close,Volume,Open Interest,strike
0,2022-09-16 09:15:00,BANKNIFTY46000CE,16-09-2022 09:15:00,13.05,13.05,12.95,12.95,100.0,34825.0,46000.0
1,2022-09-16 09:16:00,BANKNIFTY46000CE,16-09-2022 09:16:00,12.45,12.45,12.45,12.45,25.0,34825.0,46000.0
2,2022-09-16 09:17:00,BANKNIFTY46000CE,16-09-2022 09:17:00,12.50,12.50,12.40,12.40,50.0,34875.0,46000.0
3,2022-09-16 09:18:00,BANKNIFTY46000CE,16-09-2022 09:18:00,12.40,12.40,11.40,11.40,225.0,34875.0,46000.0
4,2022-09-16 09:19:00,BANKNIFTY46000CE,16-09-2022 09:19:00,12.25,12.40,12.25,12.40,75.0,34875.0,46000.0
...,...,...,...,...,...,...,...,...,...,...
2620,2022-09-22 15:25:00,BANKNIFTY46000CE,22-09-2022 15:25:00,5.75,5.80,5.35,5.40,11325.0,180525.0,46000.0
2621,2022-09-22 15:26:00,BANKNIFTY46000CE,22-09-2022 15:26:00,5.40,5.40,4.95,5.20,11525.0,180525.0,46000.0
2622,2022-09-22 15:27:00,BANKNIFTY46000CE,22-09-2022 15:27:00,5.10,5.40,5.10,5.15,11325.0,186600.0,46000.0
2623,2022-09-22 15:28:00,BANKNIFTY46000CE,22-09-2022 15:28:00,5.15,5.40,5.15,5.30,7900.0,186600.0,46000.0


In [ ]:
spot_df["datetime"] = pd.to_datetime(spot_df["Date/Time"])
spot_df = pd.merge(full_data2, spot_df, how="left").fillna(method='ffill').fillna(method='bfill')




In [ ]:
spot_df = spot_df.set_index("datetime").groupby(
  pd.Grouper(freq='15Min')).agg({
  "Open": "first",
  "High": "max",
  "Low": "min",
  "Close": "last",
  "Volume": "sum",
  "Open Interest": "last"}).dropna()

In [ ]:


cerebro = bt.Cerebro()

# spot_path = '/content/banknifty_spot.csv'
# spot_df = pd.read_csv(spot_path)

spot_data = SignalData(
        dataname=spot_df,
        timeframe=bt.TimeFrame.Minutes
)

cerebro.adddata(spot_data, name='BANKNIFTY')


for name, data in zip(keys, dfs):
  # print(name)
  # print(data)
  # data["datetime"] = pd.to_datetime(data["datetime"])
  data = SignalData(dataname=data, timeframe=bt.TimeFrame.Minutes)
  cerebro.adddata(data, name=name)


for name, data in zip(keys2, dfs2):
  # print(name)
  # print(len(data))
  # data["datetime"] = pd.to_datetime(data["datetime"])
  data = SignalData(dataname=data, timeframe=bt.TimeFrame.Minutes)
  cerebro.adddata(data, name=name)

cerebro.addstrategy(My_strategy)
cerebro.broker.setcash(1000000.0)
cerebro.broker.setcommission(commission=.0005)
# cerebro.broker = bt.brokers.BackBroker(slip_perc=0.0005)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sr')  
cerebro.addanalyzer(btanalyzers.DrawDown, _name='dd')  
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='ar')
cerebro.addanalyzer(btanalyzers.Returns, _name='r')     
cerebro.addsizer(bt.sizers.PercentSizer, percents=90)  

In [ ]:
len(cerebro.datas)

371

In [ ]:
bb = cerebro.run()
bb1 = bb[0]    
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue()) 

{'BANKNIFTY': 0, 'BANKNIFTYWK30000PE': 1, 'BANKNIFTYWK30500PE': 2, 'BANKNIFTYWK31000CE': 3, 'BANKNIFTYWK31000PE': 4, 'BANKNIFTYWK31500PE': 5, 'BANKNIFTYWK32000PE': 6, 'BANKNIFTYWK32500PE': 7, 'BANKNIFTYWK33000CE': 8, 'BANKNIFTYWK33000PE': 9, 'BANKNIFTYWK33500CE': 10, 'BANKNIFTYWK33500PE': 11, 'BANKNIFTYWK34000PE': 12, 'BANKNIFTYWK34500CE': 13, 'BANKNIFTYWK34500PE': 14, 'BANKNIFTYWK35000CE': 15, 'BANKNIFTYWK35000PE': 16, 'BANKNIFTYWK35500CE': 17, 'BANKNIFTYWK35500PE': 18, 'BANKNIFTYWK35700PE': 19, 'BANKNIFTYWK35800PE': 20, 'BANKNIFTYWK35900PE': 21, 'BANKNIFTYWK36000CE': 22, 'BANKNIFTYWK36000PE': 23, 'BANKNIFTYWK36100PE': 24, 'BANKNIFTYWK36200PE': 25, 'BANKNIFTYWK36300PE': 26, 'BANKNIFTYWK36400PE': 27, 'BANKNIFTYWK36500CE': 28, 'BANKNIFTYWK36500PE': 29, 'BANKNIFTYWK36600PE': 30, 'BANKNIFTYWK36700CE': 31, 'BANKNIFTYWK36700PE': 32, 'BANKNIFTYWK36800CE': 33, 'BANKNIFTYWK36800PE': 34, 'BANKNIFTYWK36900CE': 35, 'BANKNIFTYWK36900PE': 36, 'BANKNIFTYWK37000CE': 37, 'BANKNIFTYWK37000PE': 38, 'BAN

In [ ]:
print('Final Value: ', cerebro.broker.getvalue())
print('Sharpe Ratio:', bb1.analyzers.sr.get_analysis())
print('Drawdown:', bb1.analyzers.dd.get_analysis())
print('Annual Return:', bb1.analyzers.ar.get_analysis())
print('Returns:', bb1.analyzers.r.get_analysis())

Final Value:  1017894.0281249986
Sharpe Ratio: OrderedDict([('sharperatio', None)])
Drawdown: AutoOrderedDict([('len', 9), ('drawdown', 0.08063696474320299), ('moneydown', 821.4612500008661), ('max', AutoOrderedDict([('len', 68), ('drawdown', 0.647872981721295), ('moneydown', 6592.828125)]))])
Annual Return: OrderedDict([(2022, 0.01789402812499863)])
Returns: OrderedDict([('rtot', 0.017735814600708125), ('ravg', 0.00010134751200404642), ('rnorm', 0.00010135264783664094), ('rnorm100', 0.010135264783664093)])


In [ ]:

pyfoliozer = bb1.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

In [ ]:
import pyfolio as pf
pf.create_full_tear_sheet(returns)

/usr/local/lib/python3.9/dist-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()


Start date,2022-09-16
End date,2022-09-22
Total months,0
,Backtest
Annual return,89.4%
Cumulative returns,1.8%
Annual volatility,5.0%
Sharpe ratio,12.73
Calmar ratio,1989.76
Stability,0.72
Max drawdown,-0.0%


IndexError: ignored

In [ ]:

print(transactions)
print(returns)

                           amount   price  sid              symbol     value
date                                                                        
2022-09-16 10:15:00+00:00     -25  477.90  114  BANKNIFTYWK40900PE  11947.50
2022-09-16 10:15:00+00:00     -50  887.65  302    BANKNIFTY40600CE  44382.50
2022-09-16 10:30:00+00:00     -25  475.85  115  BANKNIFTYWK41000CE  11896.25
2022-09-16 10:30:00+00:00     -50  817.70  317    BANKNIFTY41300PE  40885.00
2022-09-16 11:00:00+00:00     -25  453.20  118  BANKNIFTYWK41100PE  11330.00
...                           ...     ...  ...                 ...       ...
2022-09-22 15:00:00+00:00     -50  617.75  311    BANKNIFTY41000PE  30887.50
2022-09-22 15:15:00+00:00      25    0.55  109  BANKNIFTYWK40700CE    -13.75
2022-09-22 15:15:00+00:00      25  177.00  112  BANKNIFTYWK40800PE  -4425.00
2022-09-22 15:15:00+00:00      50  593.60  300    BANKNIFTY40500CE -29680.00
2022-09-22 15:15:00+00:00      50  663.10  311    BANKNIFTY41000PE -33155.00

In [ ]:
var = transactions.groupby('symbol')

In [ ]:
# var.describe()

In [ ]:
grouped_trans = transactions.groupby('symbol')

In [ ]:
# grouped_trans.get_group('BANKNIFTY40600CE')

In [ ]:
journal = []
for k,v in var:
  temp = grouped_trans.get_group(k)
  # if len(temp)%2 != 0 :
  print(len(temp))
  print(temp)
  trade = []
  amt = 0
  avg_price = []
  for i in range(len(temp)):
    # print(temp["amount"][i])
    if temp["amount"][i] < 0 :
      trade.append([temp.index[i], temp["symbol"][i], temp["amount"][i], temp["price"][i]])
      amt += temp["amount"][i]
      avg_price.append(temp["price"][i])
    else :
      print(trade)
      # print(trade[0][0], temp.index[i], temp["symbol"][i], amt, np.mean(avg_price), temp["price"][i], (temp["price"][i] - np.mean(avg_price))*amt )
      journal.append([trade[0][0], temp.index[i], temp["symbol"][i], amt, np.mean(avg_price), temp["price"][i], (temp["price"][i] - np.mean(avg_price))*amt ])
      trade = []
      amt = 0
      avg_price = []
    
    # print(len(temp))


2
                           amount   price  sid            symbol    value
date                                                                     
2022-09-22 12:45:00+00:00     -50  768.45  292  BANKNIFTY40100CE  38422.5
2022-09-22 13:00:00+00:00      50  782.00  292  BANKNIFTY40100CE -39100.0
[[Timestamp('2022-09-22 12:45:00+0000', tz='UTC'), 'BANKNIFTY40100CE', -50, 768.45]]
2
                           amount   price  sid            symbol    value
date                                                                     
2022-09-22 12:15:00+00:00     -50  760.00  294  BANKNIFTY40200CE  38000.0
2022-09-22 12:30:00+00:00      50  730.05  294  BANKNIFTY40200CE -36502.5
[[Timestamp('2022-09-22 12:15:00+0000', tz='UTC'), 'BANKNIFTY40200CE', -50, 760.0]]
4
                           amount   price  sid            symbol    value
date                                                                     
2022-09-22 11:30:00+00:00     -50  768.05  296  BANKNIFTY40300CE  38402.5
2022-09-22 

IndexError: ignored

In [ ]:
j = pd.DataFrame(journal)
j.columns = ["Entry time", "Exit time", "Ticker", "Quantity", "Entry price", "Exit price", "Pnl"]
print(j)
j.to_csv("journal.csv")

                  Entry time                 Exit time            Ticker  \
0  2022-09-22 12:45:00+00:00 2022-09-22 13:00:00+00:00  BANKNIFTY40100CE   
1  2022-09-22 12:15:00+00:00 2022-09-22 12:30:00+00:00  BANKNIFTY40200CE   
2  2022-09-22 11:30:00+00:00 2022-09-22 12:00:00+00:00  BANKNIFTY40300CE   
3  2022-09-22 13:15:00+00:00 2022-09-22 14:45:00+00:00  BANKNIFTY40300CE   
4  2022-09-16 14:00:00+00:00 2022-09-16 14:45:00+00:00  BANKNIFTY40400CE   
5  2022-09-22 13:45:00+00:00 2022-09-22 14:45:00+00:00  BANKNIFTY40400CE   
6  2022-09-16 14:45:00+00:00 2022-09-16 15:15:00+00:00  BANKNIFTY40500CE   
7  2022-09-17 10:15:00+00:00 2022-09-17 15:15:00+00:00  BANKNIFTY40500CE   
8  2022-09-18 10:15:00+00:00 2022-09-18 15:15:00+00:00  BANKNIFTY40500CE   
9  2022-09-19 14:30:00+00:00 2022-09-19 14:45:00+00:00  BANKNIFTY40500CE   
10 2022-09-19 15:00:00+00:00 2022-09-19 15:15:00+00:00  BANKNIFTY40500CE   
11 2022-09-22 14:45:00+00:00 2022-09-22 15:15:00+00:00  BANKNIFTY40500CE   
12 2022-09-1

In [ ]:
net_pnl = 0
for i in j:
  # print(i[-1])
  net_pnl += i[-1]

print(net_pnl)

TypeError: ignored

In [ ]:
# fd.shape